# Objective : Build Neural network model to translate human-readable dates like into machine-readable dates

In [1]:
# Importing necessary packages

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date

C:\Users\MOHAN KUMAR SAH\.conda\envs\python3.5\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Generating Dataset

In [2]:
# Using Faker to generate dataset
# Feature : Human Readable Date
# Target : ISO format of Human Readable Date i.e., Machine Readable Date

fake = Faker()

fake.seed(101)
random.seed(101)

In [3]:
# Defining some formats for date which is commnly used by human while writing the date

FORMATS = ['short', # d/M/YY
           'medium', # MMM d, YYY
           'medium',
           'medium',
           'long', # MMMM dd, YYY
           'long',
           'long',
           'long',
           'long',
           'full', # EEEE, MMM dd, YYY
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'dd/MM/YYY',
           'EE d, MMM YYY',
           'EEEE d, MMMM YYY']

In [4]:
# Printing some formats

for format in FORMATS:
    print('%s => %s' %(format, format_date(fake.date_object(), format=format, locale='en')))

short => 7/19/09
medium => Apr 3, 1983
medium => Sep 11, 2006
medium => May 29, 1994
long => October 15, 2001
long => April 20, 1973
long => February 24, 2015
long => April 7, 2004
long => August 6, 1984
full => Monday, December 20, 2010
full => Friday, February 1, 1985
full => Sunday, August 20, 1989
d MMM YYY => 14 Jan 2003
d MMMM YYY => 13 February 2017
d MMMM YYY => 14 June 1984
d MMMM YYY => 23 May 1992
d MMMM YYY => 22 October 1999
d MMMM YYY => 15 October 1974
dd/MM/YYY => 16/05/1987
EE d, MMM YYY => Sat 26, Feb 1983
EEEE d, MMMM YYY => Wednesday 17, December 1980


random_date() will generate a random date using a random format picked from our list FORMATS defined before. It'll return a tuple with the human and machine readable date plus the date object

In [5]:
def random_date():
    obj = fake.date_object()  # date object

    try:
        # Choosing a random date of a particular format
        date = format_date(obj, format=random.choice(FORMATS), locale='en')
        
        # human readable format
        human_readable = date.lower().replace(',', '')
        
        # machne readable formats
        machine_readable = obj.isoformat()

    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, obj

create_dataset(m) will generate our dataset, taking m as the number of samples to create. It returns the dataset as a list, two dictionaries mapping index to character (these are our vocabularies), human and machine, and the inverse mapping, inv_machine, chars to index:

In [6]:
def create_dataset(m):
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    
    for i in tqdm(range(m)):
        h, m, _ = random_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    # We also add two special chars, <unk> for unknown characters, and <pad> to add padding at the end
    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v: k for k, v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine

1. dataset: generated dataset
2. human: human readable vocabulary
3. machine: machine readable vocabulary
4. inv_machine: inverse of machine readable vocabulary

In [7]:
# Generates 10k dataset
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(m)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14628.32it/s]


In [8]:
# Prints 10 recored from the dataset
dataset[:10]

[('18/01/1976', '1976-01-18'),
 ('april 18 2000', '2000-04-18'),
 ('26 july 2006', '2006-07-26'),
 ('saturday december 10 1994', '1994-12-10'),
 ('15 january 1983', '1983-01-15'),
 ('feb 5 2002', '2002-02-05'),
 ('9 march 1992', '1992-03-09'),
 ('april 2 1986', '1986-04-02'),
 ('sat 18 may 2013', '2013-05-18'),
 ('june 26 1997', '1997-06-26')]

In [9]:
# human readable vocabulary
human_vocab

{' ': 0,
 '/': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11,
 '<pad>': 35,
 '<unk>': 34,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'r': 27,
 's': 28,
 't': 29,
 'u': 30,
 'v': 31,
 'w': 32,
 'y': 33}

In [10]:
# machine readable vocabulary
machine_vocab

{'-': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10}

In [11]:
# inverse of machine readable vocabulary
inv_machine_vocab

{0: '-',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9'}

preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty) takes the whole dataset, and both human and machine vocabularie, plus some max length arguments, and it'll spit out out training set and target labels, plus the one hot encoding of both:

In [12]:
def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

1. X is training set
2. np.array(Y) is target labels
3. Xoh is one hot encoding of training set
4. Yoh is one hot encoding of target labels

string_to_int(string, length, vocab) will return a list of indexes based on a string and vocabulary, vocab, cropping or padding it depending on the max length passed in:

In [13]:
def string_to_int(string, length, vocab):
    string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep

In [14]:
string_to_int('September 10, 1978', 30, human_vocab)

[28,
 16,
 26,
 29,
 16,
 23,
 13,
 16,
 27,
 0,
 3,
 2,
 0,
 3,
 11,
 9,
 10,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35,
 35]

In [15]:
Tx = 30 # maximum length of features
Ty = 10 # maximum length of targets
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 36)
Yoh.shape: (10000, 10, 11)


# Define Model

In [16]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

one_step_attention(a, s_prev): At step  t , given all the hidden states of the Bi-LSTM ( [a<1>,a<2>,...,a<Tx>] ) and the previous hidden state of the second LSTM ( s<t−1> ), one_step_attention() will compute the attention weights ( [α<t,1>,α<t,2>,...,α<t,Tx>] ) and output the context vector:
context<t>=∑t′=0Txα<t,t′>a<t′>

In [17]:
def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [18]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation='softmax')

model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size): Implements the entire model. It first runs the input through a Bidirectional LSTM to get back  [a<1>,a<2>,...,a<Tx>] . Then, it calls one_step_attention()  Ty  times (for loop). At each iteration of this loop, it gives the computed context vector  c<t>  to the second LSTM, and runs the output of the LSTM through a dense layer with softmax activation to generate a prediction  y^<t> .

In [19]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model([X, s0, c0], outputs)
    return model

In [20]:
mod = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [21]:
mod.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 36)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17664       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

                                                                 attention_weights[4][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[5][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[6][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[7][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[8][0]          
                                                                 bidirectional_1[0][0]            
          

In [22]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [24]:
mod.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)

Epoch 1/10


 2900/10000 [=======>......................] - ETA: 17:20 - loss: 24.9550 - dense_3_loss: 2.3588 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.0200 - dense_3_acc_3: 0.1300 - dense_3_acc_4: 0.0000e+00 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0600 - dense_3_acc_7: 0.0000e+00 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.18 - ETA: 8:44 - loss: 23.5106 - dense_3_loss: 2.5579 - dense_3_acc: 0.0250 - dense_3_acc_1: 0.0650 - dense_3_acc_2: 0.0200 - dense_3_acc_3: 0.0700 - dense_3_acc_4: 0.4900 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0300 - dense_3_acc_7: 0.4950 - dense_3_acc_8: 0.0050 - dense_3_acc_9: 0.0900                     - ETA: 5:52 - loss: 22.7898 - dense_3_loss: 2.6093 - dense_3_acc: 0.1600 - dense_3_acc_1: 0.1400 - dense_3_acc_2: 0.1000 - dense_3_acc_3: 0.1033 - dense_3_acc_4: 0.3267 - dense_3_acc_5: 0.2533 - dense_3_acc_6: 0.0400 - dense_3_acc_7: 0.3333 - dense_3_acc_8: 0.1200 - dense_3_acc_9: 0.0967    - ETA: 4:26 - loss: 22.3966 - dense_3_

 5800/10000 [================>.............] - ETA: 37s - loss: 19.8183 - dense_3_loss: 2.7692 - dense_3_acc: 0.5487 - dense_3_acc_1: 0.5560 - dense_3_acc_2: 0.2880 - dense_3_acc_3: 0.1027 - dense_3_acc_4: 0.5047 - dense_3_acc_5: 0.2560 - dense_3_acc_6: 0.0353 - dense_3_acc_7: 0.7380 - dense_3_acc_8: 0.0717 - dense_3_acc_9: 0.03 - ETA: 35s - loss: 19.7476 - dense_3_loss: 2.7710 - dense_3_acc: 0.5548 - dense_3_acc_1: 0.5677 - dense_3_acc_2: 0.2910 - dense_3_acc_3: 0.1052 - dense_3_acc_4: 0.5065 - dense_3_acc_5: 0.2548 - dense_3_acc_6: 0.0352 - dense_3_acc_7: 0.7406 - dense_3_acc_8: 0.0706 - dense_3_acc_9: 0.03 - ETA: 34s - loss: 19.6707 - dense_3_loss: 2.7680 - dense_3_acc: 0.5597 - dense_3_acc_1: 0.5781 - dense_3_acc_2: 0.2953 - dense_3_acc_3: 0.1059 - dense_3_acc_4: 0.5116 - dense_3_acc_5: 0.2541 - dense_3_acc_6: 0.0353 - dense_3_acc_7: 0.7416 - dense_3_acc_8: 0.0712 - dense_3_acc_9: 0.03 - ETA: 33s - loss: 19.6059 - dense_3_loss: 2.7663 - dense_3_acc: 0.5636 - dense_3_acc_1: 0.5876 -

 8700/10000 [=========================>....] - ETA: 14s - loss: 17.9783 - dense_3_loss: 2.6548 - dense_3_acc: 0.7075 - dense_3_acc_1: 0.7446 - dense_3_acc_2: 0.3822 - dense_3_acc_3: 0.1329 - dense_3_acc_4: 0.5658 - dense_3_acc_5: 0.2273 - dense_3_acc_6: 0.0585 - dense_3_acc_7: 0.7088 - dense_3_acc_8: 0.1176 - dense_3_acc_9: 0.06 - ETA: 14s - loss: 17.9282 - dense_3_loss: 2.6504 - dense_3_acc: 0.7118 - dense_3_acc_1: 0.7485 - dense_3_acc_2: 0.3850 - dense_3_acc_3: 0.1338 - dense_3_acc_4: 0.5697 - dense_3_acc_5: 0.2252 - dense_3_acc_6: 0.0590 - dense_3_acc_7: 0.7075 - dense_3_acc_8: 0.1200 - dense_3_acc_9: 0.06 - ETA: 13s - loss: 17.8780 - dense_3_loss: 2.6475 - dense_3_acc: 0.7157 - dense_3_acc_1: 0.7518 - dense_3_acc_2: 0.3875 - dense_3_acc_3: 0.1352 - dense_3_acc_4: 0.5731 - dense_3_acc_5: 0.2249 - dense_3_acc_6: 0.0592 - dense_3_acc_7: 0.7062 - dense_3_acc_8: 0.1216 - dense_3_acc_9: 0.06 - ETA: 13s - loss: 17.8244 - dense_3_loss: 2.6430 - dense_3_acc: 0.7198 - dense_3_acc_1: 0.7553 -

10000/10000 [==============================] - ETA: 3s - loss: 16.6174 - dense_3_loss: 2.5578 - dense_3_acc: 0.7919 - dense_3_acc_1: 0.8186 - dense_3_acc_2: 0.4868 - dense_3_acc_3: 0.1432 - dense_3_acc_4: 0.6623 - dense_3_acc_5: 0.2584 - dense_3_acc_6: 0.0891 - dense_3_acc_7: 0.6638 - dense_3_acc_8: 0.1865 - dense_3_acc_9: 0.095 - ETA: 3s - loss: 16.5696 - dense_3_loss: 2.5546 - dense_3_acc: 0.7940 - dense_3_acc_1: 0.8204 - dense_3_acc_2: 0.4892 - dense_3_acc_3: 0.1436 - dense_3_acc_4: 0.6654 - dense_3_acc_5: 0.2624 - dense_3_acc_6: 0.0893 - dense_3_acc_7: 0.6657 - dense_3_acc_8: 0.1893 - dense_3_acc_9: 0.097 - ETA: 2s - loss: 16.5221 - dense_3_loss: 2.5495 - dense_3_acc: 0.7957 - dense_3_acc_1: 0.8219 - dense_3_acc_2: 0.4913 - dense_3_acc_3: 0.1432 - dense_3_acc_4: 0.6687 - dense_3_acc_5: 0.2671 - dense_3_acc_6: 0.0904 - dense_3_acc_7: 0.6678 - dense_3_acc_8: 0.1917 - dense_3_acc_9: 0.099 - ETA: 2s - loss: 16.4727 - dense_3_loss: 2.5451 - dense_3_acc: 0.7973 - dense_3_acc_1: 0.8233 - 

...

10000/10000 [==============================] - ETA: 1s - loss: 1.6446 - dense_3_loss: 0.4041 - dense_3_acc: 0.9911 - dense_3_acc_1: 0.9919 - dense_3_acc_2: 0.8994 - dense_3_acc_3: 0.9331 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.9857 - dense_3_acc_6: 0.9382 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9106 - dense_3_acc_9: 0.88 - ETA: 1s - loss: 1.6439 - dense_3_loss: 0.4046 - dense_3_acc: 0.9912 - dense_3_acc_1: 0.9920 - dense_3_acc_2: 0.9001 - dense_3_acc_3: 0.9333 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.9857 - dense_3_acc_6: 0.9384 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9099 - dense_3_acc_9: 0.87 - ETA: 1s - loss: 1.6406 - dense_3_loss: 0.4036 - dense_3_acc: 0.9913 - dense_3_acc_1: 0.9921 - dense_3_acc_2: 0.9004 - dense_3_acc_3: 0.9327 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.9858 - dense_3_acc_6: 0.9390 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9102 - dense_3_acc_9: 0.88 - ETA: 1s - loss: 1.6392 - dense_3_loss: 0.4029 - dense_3_acc: 0.9913 - dense_3_acc_1: 0.9921 - dense_3

 2900/10000 [=======>......................] - ETA: 16s - loss: 1.5158 - dense_3_loss: 0.3587 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9000 - dense_3_acc_3: 0.9700 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 0.9400 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.8800 - dense_3_acc_9: 0.920 - ETA: 16s - loss: 1.5289 - dense_3_loss: 0.3681 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9100 - dense_3_acc_3: 0.9500 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9950 - dense_3_acc_6: 0.9300 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9100 - dense_3_acc_9: 0.905 - ETA: 16s - loss: 1.4839 - dense_3_loss: 0.3766 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 0.9233 - dense_3_acc_3: 0.9567 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.9967 - dense_3_acc_6: 0.9467 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9233 - dense_3_acc_9: 0.906 - ETA: 15s - loss: 1.4437 - dense_3_loss: 0.3621 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - 

 5800/10000 [================>.............] - ETA: 11s - loss: 1.4683 - dense_3_loss: 0.3558 - dense_3_acc: 0.9927 - dense_3_acc_1: 0.9937 - dense_3_acc_2: 0.9060 - dense_3_acc_3: 0.9460 - dense_3_acc_4: 0.9997 - dense_3_acc_5: 0.9860 - dense_3_acc_6: 0.9500 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9260 - dense_3_acc_9: 0.904 - ETA: 10s - loss: 1.4670 - dense_3_loss: 0.3558 - dense_3_acc: 0.9926 - dense_3_acc_1: 0.9935 - dense_3_acc_2: 0.9061 - dense_3_acc_3: 0.9458 - dense_3_acc_4: 0.9997 - dense_3_acc_5: 0.9861 - dense_3_acc_6: 0.9500 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9261 - dense_3_acc_9: 0.903 - ETA: 10s - loss: 1.4690 - dense_3_loss: 0.3563 - dense_3_acc: 0.9922 - dense_3_acc_1: 0.9928 - dense_3_acc_2: 0.9053 - dense_3_acc_3: 0.9453 - dense_3_acc_4: 0.9997 - dense_3_acc_5: 0.9859 - dense_3_acc_6: 0.9497 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9266 - dense_3_acc_9: 0.903 - ETA: 10s - loss: 1.4723 - dense_3_loss: 0.3593 - dense_3_acc: 0.9921 - dense_3_acc_1: 0.9927 - 

 8700/10000 [=========================>....] - ETA: 6s - loss: 1.4687 - dense_3_loss: 0.3647 - dense_3_acc: 0.9922 - dense_3_acc_1: 0.9927 - dense_3_acc_2: 0.9056 - dense_3_acc_3: 0.9459 - dense_3_acc_4: 0.9998 - dense_3_acc_5: 0.9858 - dense_3_acc_6: 0.9497 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9263 - dense_3_acc_9: 0.89 - ETA: 6s - loss: 1.4663 - dense_3_loss: 0.3638 - dense_3_acc: 0.9922 - dense_3_acc_1: 0.9927 - dense_3_acc_2: 0.9050 - dense_3_acc_3: 0.9462 - dense_3_acc_4: 0.9998 - dense_3_acc_5: 0.9858 - dense_3_acc_6: 0.9503 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9263 - dense_3_acc_9: 0.89 - ETA: 6s - loss: 1.4664 - dense_3_loss: 0.3630 - dense_3_acc: 0.9920 - dense_3_acc_1: 0.9925 - dense_3_acc_2: 0.9056 - dense_3_acc_3: 0.9464 - dense_3_acc_4: 0.9998 - dense_3_acc_5: 0.9854 - dense_3_acc_6: 0.9498 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9266 - dense_3_acc_9: 0.89 - ETA: 5s - loss: 1.4701 - dense_3_loss: 0.3642 - dense_3_acc: 0.9919 - dense_3_acc_1: 0.9924 - dense_3

10000/10000 [==============================] - ETA: 1s - loss: 1.4722 - dense_3_loss: 0.3605 - dense_3_acc: 0.9914 - dense_3_acc_1: 0.9924 - dense_3_acc_2: 0.9044 - dense_3_acc_3: 0.9473 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.9858 - dense_3_acc_6: 0.9469 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9220 - dense_3_acc_9: 0.89 - ETA: 1s - loss: 1.4719 - dense_3_loss: 0.3604 - dense_3_acc: 0.9913 - dense_3_acc_1: 0.9924 - dense_3_acc_2: 0.9042 - dense_3_acc_3: 0.9472 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.9857 - dense_3_acc_6: 0.9466 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9220 - dense_3_acc_9: 0.89 - ETA: 1s - loss: 1.4743 - dense_3_loss: 0.3606 - dense_3_acc: 0.9910 - dense_3_acc_1: 0.9920 - dense_3_acc_2: 0.9039 - dense_3_acc_3: 0.9470 - dense_3_acc_4: 0.9999 - dense_3_acc_5: 0.9858 - dense_3_acc_6: 0.9467 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.9217 - dense_3_acc_9: 0.89 - ETA: 1s - loss: 1.4738 - dense_3_loss: 0.3606 - dense_3_acc: 0.9909 - dense_3_acc_1: 0.9919 - dense_3

In [25]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab) # Converts string to list of integers
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = mod.predict([source, s0, c0]) # Prediction
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("Input: ", example)
    print("Output:", ''.join(output))
    print()

Input:  3 May 1979
Output: 1999-05-03

Input:  5 April 09
Output: 2009-04-05

Input:  21th of August 2016
Output: 2016-08-11

Input:  Tue 10 Jul 2007
Output: 2007-07-10

Input:  Saturday May 9 2018
Output: 2018-05-09

Input:  March 3 2001
Output: 2011-03-03

Input:  March 3rd 2001
Output: 2001-03-03

Input:  1 March 2001
Output: 2001-03-01



# Full Code

In [27]:
# Importing necessary packages
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date

# Using Faker to generate dataset
# Feature : Human Readable Date
# Target : ISO format of Human Readable Date i.e., Machine Readable Date
fake = Faker()
fake.seed(101)
random.seed(101)

# Defining some formats for date which is commnly used by human while writing the date
FORMATS = ['short', # d/M/YY
           'medium', # MMM d, YYY
           'medium',
           'medium',
           'long', # MMMM dd, YYY
           'long',
           'long',
           'long',
           'long',
           'full', # EEEE, MMM dd, YYY
           'full',
           'full',
           'd MMM YYY', 
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'd MMMM YYY',
           'dd/MM/YYY',
           'EE d, MMM YYY',
           'EEEE d, MMMM YYY']


def random_date():
    obj = fake.date_object()  # date object

    try:
        # Choosing a random date of a particular format
        date = format_date(obj, format=random.choice(FORMATS), locale='en')
        
        # human readable format
        human_readable = date.lower().replace(',', '')
        
        # machne readable formats
        machine_readable = obj.isoformat()

    except AttributeError as e:
        return None, None, None

    return human_readable, machine_readable, obj


def create_dataset(m):
    human_vocab = set()
    machine_vocab = set()
    dataset = []
    
    for i in tqdm(range(m)):
        h, m, _ = random_date()
        if h is not None:
            dataset.append((h, m))
            human_vocab.update(tuple(h))
            machine_vocab.update(tuple(m))
    
    # We also add two special chars, <unk> for unknown characters, and <pad> to add padding at the end
    human = dict(zip(sorted(human_vocab) + ['<unk>', '<pad>'], list(range(len(human_vocab) + 2))))
    inv_machine = dict(enumerate(sorted(machine_vocab)))
    machine = {v: k for k, v in inv_machine.items()}
 
    return dataset, human, machine, inv_machine


# Generates 20k dataset
m = 20000
dataset, human_vocab, machine_vocab, inv_machine_vocab = create_dataset(m)


def preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*dataset)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh


def string_to_int(string, length, vocab):
    string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep


Tx = 30 # maximum length of features
Ty = 10 # maximum length of targets
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)


# Define model
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

def one_step_attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

n_a = 32 # 32 memory units for bidirectinoal LSTM Layer
n_s = 64 # 64 memory units for post activation LSTM Layer
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation='softmax')


def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model([X, s0, c0], outputs)
    return model


mod = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))


mod.fit([Xoh, s0, c0], outputs, epochs=10, batch_size=100)



def UserInput(example):    
    source = string_to_int(example, Tx, human_vocab) # Converts string to list of integers
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = source.reshape((1, ) + source.shape)
    prediction = mod.predict([source, s0, c0]) # Prediction
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    return ''.join(output)
print("########################################################################")

while(True):
    print("Enter 1 for input and 0 for exit")
    n = int(input("Enter your choice: "))
    if n==1:
        example = input("Enter a Human Readable Date: ")
        Output = UserInput(example)
        print("Machine Readable Date: ", Output)
    elif n==0:
        print("Programme Ended...!")
        break
    else:
        print("Wrong entry")

100%|█████████████████████████████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 14757.95it/s]


Epoch 1/10


 2900/20000 [===>..........................] - ETA: 39:05 - loss: 25.2986 - dense_6_loss: 2.5599 - dense_6_acc: 0.0000e+00 - dense_6_acc_1: 0.0000e+00 - dense_6_acc_2: 0.0000e+00 - dense_6_acc_3: 0.0900 - dense_6_acc_4: 0.0000e+00 - dense_6_acc_5: 0.0000e+00 - dense_6_acc_6: 0.0700 - dense_6_acc_7: 0.0000e+00 - dense_6_acc_8: 0.0000e+00 - dense_6_acc_9: 0.08 - ETA: 19:44 - loss: 23.6869 - dense_6_loss: 2.6262 - dense_6_acc: 0.0000e+00 - dense_6_acc_1: 0.1800 - dense_6_acc_2: 0.1050 - dense_6_acc_3: 0.1000 - dense_6_acc_4: 0.0200 - dense_6_acc_5: 0.3650 - dense_6_acc_6: 0.0800 - dense_6_acc_7: 0.0100 - dense_6_acc_8: 0.1500 - dense_6_acc_9: 0.0800                       - ETA: 13:17 - loss: 23.1532 - dense_6_loss: 2.7719 - dense_6_acc: 0.0000e+00 - dense_6_acc_1: 0.1367 - dense_6_acc_2: 0.0733 - dense_6_acc_3: 0.0667 - dense_6_acc_4: 0.3333 - dense_6_acc_5: 0.2533 - dense_6_acc_6: 0.0567 - dense_6_acc_7: 0.3267 - dense_6_acc_8: 0.1067 - dense_6_acc_9: 0.05 - ETA: 10:07 - loss: 22.7150 - 

 5800/20000 [=======>......................] - ETA: 1:39 - loss: 20.0605 - dense_6_loss: 2.8180 - dense_6_acc: 0.4627 - dense_6_acc_1: 0.4657 - dense_6_acc_2: 0.2697 - dense_6_acc_3: 0.0903 - dense_6_acc_4: 0.5867 - dense_6_acc_5: 0.2107 - dense_6_acc_6: 0.0507 - dense_6_acc_7: 0.7537 - dense_6_acc_8: 0.0730 - dense_6_acc_9: 0.031 - ETA: 1:36 - loss: 19.9882 - dense_6_loss: 2.8150 - dense_6_acc: 0.4645 - dense_6_acc_1: 0.4761 - dense_6_acc_2: 0.2735 - dense_6_acc_3: 0.0906 - dense_6_acc_4: 0.5919 - dense_6_acc_5: 0.2087 - dense_6_acc_6: 0.0513 - dense_6_acc_7: 0.7545 - dense_6_acc_8: 0.0729 - dense_6_acc_9: 0.032 - ETA: 1:34 - loss: 19.9072 - dense_6_loss: 2.8108 - dense_6_acc: 0.4709 - dense_6_acc_1: 0.4853 - dense_6_acc_2: 0.2747 - dense_6_acc_3: 0.0894 - dense_6_acc_4: 0.5975 - dense_6_acc_5: 0.2056 - dense_6_acc_6: 0.0513 - dense_6_acc_7: 0.7572 - dense_6_acc_8: 0.0731 - dense_6_acc_9: 0.032 - ETA: 1:32 - loss: 19.8290 - dense_6_loss: 2.8099 - dense_6_acc: 0.4782 - dense_6_acc_1: 0

 8700/20000 [============>.................] - ETA: 56s - loss: 17.9764 - dense_6_loss: 2.6672 - dense_6_acc: 0.6708 - dense_6_acc_1: 0.6866 - dense_6_acc_2: 0.3698 - dense_6_acc_3: 0.1161 - dense_6_acc_4: 0.6669 - dense_6_acc_5: 0.1729 - dense_6_acc_6: 0.0761 - dense_6_acc_7: 0.7017 - dense_6_acc_8: 0.1517 - dense_6_acc_9: 0.07 - ETA: 55s - loss: 17.9141 - dense_6_loss: 2.6632 - dense_6_acc: 0.6758 - dense_6_acc_1: 0.6917 - dense_6_acc_2: 0.3758 - dense_6_acc_3: 0.1167 - dense_6_acc_4: 0.6708 - dense_6_acc_5: 0.1713 - dense_6_acc_6: 0.0765 - dense_6_acc_7: 0.6992 - dense_6_acc_8: 0.1557 - dense_6_acc_9: 0.08 - ETA: 55s - loss: 17.8569 - dense_6_loss: 2.6585 - dense_6_acc: 0.6803 - dense_6_acc_1: 0.6961 - dense_6_acc_2: 0.3798 - dense_6_acc_3: 0.1162 - dense_6_acc_4: 0.6741 - dense_6_acc_5: 0.1695 - dense_6_acc_6: 0.0775 - dense_6_acc_7: 0.6943 - dense_6_acc_8: 0.1608 - dense_6_acc_9: 0.08 - ETA: 54s - loss: 17.7917 - dense_6_loss: 2.6519 - dense_6_acc: 0.6850 - dense_6_acc_1: 0.7005 -

...

 8700/20000 [============>.................] - ETA: 23s - loss: 0.8918 - dense_6_loss: 0.2169 - dense_6_acc: 0.9941 - dense_6_acc_1: 0.9937 - dense_6_acc_2: 0.9103 - dense_6_acc_3: 0.9761 - dense_6_acc_4: 1.0000 - dense_6_acc_5: 0.9890 - dense_6_acc_6: 0.9561 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9734 - dense_6_acc_9: 0.944 - ETA: 23s - loss: 0.8905 - dense_6_loss: 0.2161 - dense_6_acc: 0.9942 - dense_6_acc_1: 0.9938 - dense_6_acc_2: 0.9103 - dense_6_acc_3: 0.9760 - dense_6_acc_4: 1.0000 - dense_6_acc_5: 0.9892 - dense_6_acc_6: 0.9558 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9733 - dense_6_acc_9: 0.944 - ETA: 22s - loss: 0.8880 - dense_6_loss: 0.2153 - dense_6_acc: 0.9939 - dense_6_acc_1: 0.9938 - dense_6_acc_2: 0.9110 - dense_6_acc_3: 0.9762 - dense_6_acc_4: 1.0000 - dense_6_acc_5: 0.9892 - dense_6_acc_6: 0.9564 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9733 - dense_6_acc_9: 0.944 - ETA: 22s - loss: 0.8830 - dense_6_loss: 0.2143 - dense_6_acc: 0.9940 - dense_6_acc_1: 0.9939 - 

11600/20000 [================>.............] - ETA: 18s - loss: 0.8881 - dense_6_loss: 0.2136 - dense_6_acc: 0.9935 - dense_6_acc_1: 0.9936 - dense_6_acc_2: 0.9105 - dense_6_acc_3: 0.9756 - dense_6_acc_4: 1.0000 - dense_6_acc_5: 0.9892 - dense_6_acc_6: 0.9570 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9745 - dense_6_acc_9: 0.944 - ETA: 18s - loss: 0.8900 - dense_6_loss: 0.2139 - dense_6_acc: 0.9934 - dense_6_acc_1: 0.9936 - dense_6_acc_2: 0.9099 - dense_6_acc_3: 0.9752 - dense_6_acc_4: 1.0000 - dense_6_acc_5: 0.9892 - dense_6_acc_6: 0.9567 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9748 - dense_6_acc_9: 0.944 - ETA: 18s - loss: 0.8904 - dense_6_loss: 0.2134 - dense_6_acc: 0.9932 - dense_6_acc_1: 0.9936 - dense_6_acc_2: 0.9101 - dense_6_acc_3: 0.9746 - dense_6_acc_4: 1.0000 - dense_6_acc_5: 0.9891 - dense_6_acc_6: 0.9566 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9749 - dense_6_acc_9: 0.944 - ETA: 17s - loss: 0.8913 - dense_6_loss: 0.2138 - dense_6_acc: 0.9933 - dense_6_acc_1: 0.9936 - 

14500/20000 [====================>.........] - ETA: 13s - loss: 0.8931 - dense_6_loss: 0.2172 - dense_6_acc: 0.9935 - dense_6_acc_1: 0.9932 - dense_6_acc_2: 0.9110 - dense_6_acc_3: 0.9746 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9891 - dense_6_acc_6: 0.9571 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9744 - dense_6_acc_9: 0.942 - ETA: 13s - loss: 0.8938 - dense_6_loss: 0.2171 - dense_6_acc: 0.9933 - dense_6_acc_1: 0.9931 - dense_6_acc_2: 0.9109 - dense_6_acc_3: 0.9745 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9891 - dense_6_acc_6: 0.9568 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9744 - dense_6_acc_9: 0.943 - ETA: 13s - loss: 0.8933 - dense_6_loss: 0.2167 - dense_6_acc: 0.9933 - dense_6_acc_1: 0.9931 - dense_6_acc_2: 0.9105 - dense_6_acc_3: 0.9745 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9891 - dense_6_acc_6: 0.9569 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9744 - dense_6_acc_9: 0.943 - ETA: 13s - loss: 0.8950 - dense_6_loss: 0.2171 - dense_6_acc: 0.9933 - dense_6_acc_1: 0.9930 - 

17400/20000 [=========================>....] - ETA: 8s - loss: 0.8899 - dense_6_loss: 0.2138 - dense_6_acc: 0.9932 - dense_6_acc_1: 0.9934 - dense_6_acc_2: 0.9098 - dense_6_acc_3: 0.9743 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9886 - dense_6_acc_6: 0.9571 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9741 - dense_6_acc_9: 0.94 - ETA: 8s - loss: 0.8895 - dense_6_loss: 0.2136 - dense_6_acc: 0.9933 - dense_6_acc_1: 0.9935 - dense_6_acc_2: 0.9099 - dense_6_acc_3: 0.9743 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9886 - dense_6_acc_6: 0.9571 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9740 - dense_6_acc_9: 0.94 - ETA: 8s - loss: 0.8897 - dense_6_loss: 0.2138 - dense_6_acc: 0.9932 - dense_6_acc_1: 0.9934 - dense_6_acc_2: 0.9097 - dense_6_acc_3: 0.9743 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9886 - dense_6_acc_6: 0.9572 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9739 - dense_6_acc_9: 0.94 - ETA: 8s - loss: 0.8903 - dense_6_loss: 0.2142 - dense_6_acc: 0.9932 - dense_6_acc_1: 0.9934 - dense_6

20000/20000 [==============================] - ETA: 4s - loss: 0.8821 - dense_6_loss: 0.2124 - dense_6_acc: 0.9930 - dense_6_acc_1: 0.9934 - dense_6_acc_2: 0.9111 - dense_6_acc_3: 0.9753 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9887 - dense_6_acc_6: 0.9582 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9741 - dense_6_acc_9: 0.94 - ETA: 3s - loss: 0.8823 - dense_6_loss: 0.2122 - dense_6_acc: 0.9929 - dense_6_acc_1: 0.9934 - dense_6_acc_2: 0.9110 - dense_6_acc_3: 0.9753 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9887 - dense_6_acc_6: 0.9582 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9742 - dense_6_acc_9: 0.94 - ETA: 3s - loss: 0.8811 - dense_6_loss: 0.2118 - dense_6_acc: 0.9929 - dense_6_acc_1: 0.9934 - dense_6_acc_2: 0.9112 - dense_6_acc_3: 0.9754 - dense_6_acc_4: 0.9999 - dense_6_acc_5: 0.9886 - dense_6_acc_6: 0.9581 - dense_6_acc_7: 1.0000 - dense_6_acc_8: 0.9743 - dense_6_acc_9: 0.94 - ETA: 3s - loss: 0.8817 - dense_6_loss: 0.2117 - dense_6_acc: 0.9929 - dense_6_acc_1: 0.9934 - dense_6

# Process to run the code...!!!

1. Run the Cell having Full Code
2. Wait for training the model
3. After successful training it will ask you a choice whether you want to give input or not
4. Enter your choice
5. Accoding to you choice it will shows the output